In [5]:
import sys
sys.path.insert(0, r"K:/Thesis/codes/crypto_project")
sys.path.insert(0, r"E:/Thesis/crypto_project")
import os
import sqlite3
import pandas as pd

DATABASE_LOCATION = r"E:/Thesis/database"
from factor_model.model_update.database_generators import (
    RAW_DATA_DB,
    RETURN_DB,
    FACTOR_MODEL_ESTIMATES
)
from typing import Dict


In [60]:
from factor_model.risk_calculations.factor_covariance import (
    get_factor_return_correlation,
    get_factor_return_standard_deviation
)


#### 0. Parameters

In [15]:
risk_calculation_parameters = {
    "correlation_half_life": 730,  # days
    "variance_half_life": 365,  # days
    "specific_risk_half_life": 365,
    "date": "2023-03-04"
}

#### 1. Load the factor returns

In [17]:
with sqlite3.connect(os.path.join(DATABASE_LOCATION, FACTOR_MODEL_ESTIMATES)) as conn:
    factor_returns = pd.read_sql_query("SELECT * FROM factor_returns", conn)

#### 2. factor covariance related calculations

In [61]:
correlation = get_factor_return_correlation(factor_returns, risk_calculation_parameters)
display(correlation)

market  momentum  new_coin  reversal      size    volume
1765 market    1.000000  0.020237 -0.227545 -0.048430  0.303360 -0.730431
     momentum  0.020237  1.000000  0.082769 -0.265184 -0.097476  0.053568
     new_coin -0.227545  0.082769  1.000000  0.104668  0.037091  0.406697
     reversal -0.048430 -0.265184  0.104668  1.000000  0.027737  0.041279
     size      0.303360 -0.097476  0.037091  0.027737  1.000000 -0.592027
     volume   -0.730431  0.053568  0.406697  0.041279 -0.592027  1.000000

In [62]:
parameters = risk_calculation_parameters
NON_FACTOR_COLUMNS = ["id", "date", "version_date"]


In [73]:
std = get_factor_return_standard_deviation(factor_returns, risk_calculation_parameters)


In [75]:
factor_returns = pd.DataFrame({
    "date": ["2024-03-08", "2024-03-09", "2024-03-10"],
    "factor1": [0.02, 0.03, 0.01],
    "factor2": [0.015, 0.0125, 0.03212]
})

parameters = {
    "date": "2024-03-09",
    "variance_half_life": 3
}

result = get_factor_return_standard_deviation(factor_returns, parameters)


In [76]:
result

factor1    0.007071
factor2    0.007071
dtype: float64

In [57]:
factor_returns_data3 = pd.DataFrame(
            {
                "date": ["2024-03-01", "2024-03-02", "2024-03-03"],
                "factor1": [0.01, 0.02, 0.03],
                "factor2": [-0.02, 0.01, 0.02],
            })

In [59]:
params = {"date": "2024-03-03", "correlation_half_life": 0.001}
result = get_factor_return_correlation(factor_returns_data3, params)

In [54]:
result

factor1   factor2
2 factor1  1.000000  0.959901
  factor2  0.959901  1.000000

In [56]:
abs(result["factor2"].values[0] - 0.95990131006)<0.000001 

True